# TensorFlow Basics

In [1]:
import tensorflow as tf

ImportError: bad magic number in 'tensorflow': b'\x03\xf3\r\n'

### 3 # a rank 0 tensor; this is a scalar with shape [] <BR> [1. ,2., 3.] # a rank 1 tensor; this is a vector with shape [3] <br> [[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3] <BR> [[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

  You might think of TensorFlow Core programs as consisting of two discrete sections:

  Building the computational graph.
  
  Running the computational graph.


In [11]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const_12:0", shape=(), dtype=float32) Tensor("Const_13:0", shape=(), dtype=float32)


### To actually evaluate the nodes, we must run the computational graph within a session. A session encapsulates the control and state of the TensorFlow runtime.

In [15]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [16]:
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ",sess.run(node3))

node3:  Tensor("Add_2:0", shape=(), dtype=float32)
sess.run(node3):  7.0


### TensorBoard is a suite of web applications for inspecting and understanding your TensorFlow runs and graphs. TensorBoard currently supports five visualizations: scalars, images, audio, histograms, and the graph.

In [18]:
# Note that I did mkdir /tmp/logs first
tf.summary.FileWriter('/tmp/logs', sess.graph);

  tensorboard --logdir=/tmp/logs
  
  Now go to http://localhost:6006 in a web browser

### A graph can be paramaterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later.

In [6]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

In [7]:
print(sess.run(adder_node, {a: 3, b:4.5}))
print(sess.run(adder_node, {a: [1,3], b: [2, 4]}))

7.5
[ 3.  7.]


In [8]:
tf.summary.FileWriter('/tmp/logs', sess.graph);

In [9]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b:4.5}))

22.5


In [1]:
tf.summary.FileWriter('/tmp/logs', sess.graph);

NameError: name 'tf' is not defined

### Variables allow us to add trainable parameters to a graph. They are constructed with a type and initial value:

In [11]:
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

### Constants are initialized when you call tf.constant, and their value can never change. By contrast, variables are not initialized when you call tf.Variable. To initialize all the variables in a TensorFlow program, you must explicitly call a special operation as follows:

In [12]:
init = tf.global_variables_initializer()
sess.run(init)

In [13]:
print(sess.run(linear_model, {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


### A loss function measures how far apart the current model is from the provided data. We'll use a standard loss model for linear regression, which sums the squares of the deltas between the current model and the provided data. linear_model - y creates a vector where each element is the corresponding example's error delta. We call tf.square to square that error. Then, we sum all the squared errors to create a single scalar that abstracts the error of all examples using tf.reduce_sum:

In [14]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

23.66


In [15]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


### TensorFlow provides optimizers that slowly change each variable in order to minimize the loss function. The simplest optimizer is gradient descent. It modifies each variable according to the magnitude of the derivative of loss with respect to that variable.

In [16]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


In [36]:
tf.summary.FileWriter('/tmp/logs', sess.graph);

# Learning a more complex model

### These two lines of code below will download and read in the data automatically

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


#### The MNIST data is split into three parts: 55,000 data points of training data (mnist.train), 10,000 points of test data (mnist.test), and 5,000 points of validation data (mnist.validation). 
#### The training images are mnist.train.images and the training labels are mnist.train.labels.  Each image is 28x28 = 784.
#### mnist.train.images is a tensor (an n-dimensional array) with a shape of [55000, 784]
#### mnist.train.labels is a [55000, 10] array of floats with a one-hot encoding of the correct class label

![alt text][img]

[img]: https://www.tensorflow.org/images/softmax-regression-scalargraph.png

# What is the softmax? 

![alt text][img]

[img]: https://www.tensorflow.org/images/softmax-regression-scalarequation.png

![alt text][img]

[img]: https://www.tensorflow.org/images/softmax-regression-vectorequation.png

In [5]:
x = tf.placeholder(tf.float32, [None, 784])

#### The None means that the dimension can be of any length.  In this case, the number of training examples.

In [6]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [7]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

### That's the model!  Note we compute xW instead of Wx so we get the right ouput dimension.

### Now on to the loss.  We'll use cross entropy to compare the one hot true value with the softmax values produced by the model.

In [8]:
y_ = tf.placeholder(tf.float32, [None, 10])

In [9]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

#### reduction index says to reduce over columns or rows

In [10]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [11]:
sess = tf.InteractiveSession()

In [12]:
tf.global_variables_initializer().run()

In [13]:
for _ in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [14]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [15]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [16]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9203


In [17]:
tf.summary.FileWriter('/tmp/logs', sess.graph);